# Photography Learning Assistant with Llama-3.2

In this notebook, I have utilized **meta-llama/Llama-3.2-1B-Instruct** model to develop a photography learning assistant designed to support amateur photographers. This assistant is capable of performing two key functions:

1. **Answering Photography-Related Questions**: The assistant can respond to inquiries about various photography topics.
2. **Generating Photography Exercises**: It can create exercises to help you improve your skills across five different subjects:
   - **Composition**
   - **Lighting**
   - **Camera Settings**
   - **Exposure**
   - **Post-Processing**


### Install and Upgrade the Transformers Library

The **Transformers** library, developed by Hugging Face, is an open-source natural language processing (NLP) library that provides pre-trained models for a variety of tasks, including text classification, translation, question answering, and more. It supports multiple architectures, such as BERT, GPT-2, and T5, making it easy to integrate state-of-the-art machine learning models into applications.

This ensure you have the latest features and improvements for natural language processing tasks. Also this official page also recommends upgrading https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


### Request access to the newly launched model

Once you gated access is approved, retrieve your access token from your HF account and store it securely in Google Colab (tip - look for the key shaped icon the left most column).

In Google Colab, you can securely manage user data such as tokens or API keys using the `userdata` module. The following code demonstrates how to retrieve a secret key stored in the user data:

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

In [ ]:
#To use PyTorch in our code, we need to import the torch library
import torch

In [ ]:
# From the transformers library, we import the `AutoTokenizer` and `AutoModelForCausalLM` classes and initialize a pre-trained model and its corresponding tokenizer

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B-Instruct"  # or "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name) # tokenizer enables preprocessing of input text
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

 ### Q & A function that generates an answer to a photography-related question:

Functionality includes:
- Input Preparation (via tokenizer)
- Text Generation (via model.generate method)
- Output Decoding (via tokenizer.decode method)


In [ ]:
def answer_question(question, max_tokens=300):
    """Generate an answer to a given question about photography."""
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### Testing of Q & A Function

Testing with sample questions helps you refine your user experience

In [ ]:
# Test the function with a sample question
sample_question = "What are different types of photography?"
answer = answer_question(sample_question)
print("Question:", sample_question)
print("Answer:", answer)

Question: What are different types of photography?
Answer: What are different types of photography??
Photography is a broad term that encompasses various techniques, styles, and approaches to capturing images. Here are some of the different types of photography:

1. **Portrait Photography**: Focuses on capturing the personality, expression, and character of a person. It can be formal or informal, and may involve posing, lighting, and composition.
2. **Landscape Photography**: Depicts the natural world, such as mountains, oceans, and sunsets. It often involves capturing the play of light and shadow on the subject.
3. **Street Photography**: Documents everyday life and scenes in public places, such as streets, markets, and parks. It often involves capturing candid moments and interactions between people.
4. **Wildlife Photography**: Captures images of animals in their natural habitats, often using specialized equipment and techniques. It may involve long-term observation and patience.
5.

### Photography Exercise function that generates a photo assigment

A photography assignment is generated based on a predefined subject list of ["Composition", "Lighting", "Camera Settings", "Exposure", "Post-Processing"].

Based on the selection of 1 of the 5 topics, the LLM will provide an assignment that includes objective, materials needed, steps and expected outcome.


In [ ]:
photography_subjects = ["Composition", "Lighting", "Camera Settings", "Exposure", "Post-Processing"]

In [ ]:
def generate_practice(subject, max_length=400, temperature=0.7, top_p=0.9):
    """
    Generate a concise photography exercise for a given subject.

    Parameters:
    - subject (str): The photography topic for which to create an exercise.
    - max_length (int): Maximum number of tokens for the generated output (default: 300).
    - temperature (float): Sampling temperature for creativity (default: 0.7).
    - top_p (float): Nucleus sampling parameter for diversity (default: 0.9).

    Returns:
    - str: A concise exercise description for the given photography subject.
    """
    if not subject:
        raise ValueError("Subject must be a non-empty string.")

    prompt = (
        f"Create a concise photography exercise for {subject}. "
        f"The exercise should include brief components: "
        f"1. Objective: One sentence about what students should learn. "
        f"2. Materials needed: List essential equipment or resources (bullet points). "
        f"3. Steps: Three to four concise instructions to complete the exercise. "
        f"4. Expected outcomes: One sentence on what students should achieve."
    )

    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate output with adjusted parameters
    outputs = model.generate(
        **inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_length=max_length,
        temperature=temperature,
        top_p=top_p
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


### Testing of 'Photography Exercise' Function

Testing with sample inputs helps you refine your user experience better

In [ ]:
# Define a sample subject
sample_subject = "Composition"

# Call the function and store the output
practice_exercise = generate_practice(sample_subject)

# Print the input and output
print("Sample Input (Subject):", sample_subject)
print("Generated Practice Exercise:", practice_exercise)

Sample Input (Subject): Composition
Generated Practice Exercise: Create a concise photography exercise for Composition. The exercise should include brief components: 1. Objective: One sentence about what students should learn. 2. Materials needed: List essential equipment or resources (bullet points). 3. Steps: Three to four concise instructions to complete the exercise. 4. Expected outcomes: One sentence on what students should achieve. 5. Tips and variations: Brief description of how to adapt the exercise to suit different learning styles or circumstances.

**Exercise:** "Moonlit Garden"

**Objective:** To practice composition techniques to capture the beauty of a moonlit garden, focusing on leading the viewer's eye through the scene.

**Materials needed:**

* A smartphone or camera with a good camera
* A tripod or stabilizer (optional)
* A flashlight or lantern (to simulate moonlight)
* A notebook and pencil for note-taking
* A photo editing software (optional)

**Steps:**

1. Find 